# Backtest Execution

Run a backtest for a single strategy and view results.

## Configuration

Set the strategy name and date range below.


In [ ]:
# Configuration
strategy_name = 'spy_sma_cross'  # Change this to your strategy name
start_date = '2020-01-01'  # Start date (YYYY-MM-DD)
end_date = None  # None = today
capital_base = None  # None = from config
bundle = None  # None = auto-detect
asset_class = None  # None = auto-detect


In [ ]:
# Setup: Add project root to path
import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Imports
from lib.backtest import run_backtest, save_results
from lib.config import load_strategy_params
import pandas as pd
import json


In [ ]:
# Load strategy parameters
try:
    params = load_strategy_params(strategy_name, asset_class)
    print(f"✓ Loaded parameters for {strategy_name}")
except FileNotFoundError as e:
    print(f"✗ Error: {e}")
    raise


In [ ]:
# Run backtest
print(f"Running backtest for {strategy_name}...")
print(f"Date range: {start_date} to {end_date or 'today'}")

perf = run_backtest(
    strategy_name=strategy_name,
    start_date=start_date,
    end_date=end_date,
    capital_base=capital_base,
    bundle=bundle,
    asset_class=asset_class
)

print("✓ Backtest complete")


In [ ]:
# Save results
result_dir = save_results(
    strategy_name=strategy_name,
    perf=perf,
    params=params,
    result_type='backtest'
)

print(f"✓ Results saved to: {result_dir}")


In [ ]:
# Display metrics
metrics_file = result_dir / 'metrics.json'
with open(metrics_file) as f:
    metrics = json.load(f)

print("=" * 60)
print("PERFORMANCE METRICS")
print("=" * 60)
print(f"Total Return: {metrics.get('total_return', 0):.2%}")
print(f"Annual Return: {metrics.get('annual_return', 0):.2%}")
print(f"Sharpe Ratio: {metrics.get('sharpe', 0):.3f}")
print(f"Sortino Ratio: {metrics.get('sortino', 0):.3f}")
print(f"Max Drawdown: {metrics.get('max_drawdown', 0):.2%}")
print(f"Calmar Ratio: {metrics.get('calmar', 0):.3f}")
print(f"Annual Volatility: {metrics.get('annual_volatility', 0):.2%}")

if 'trade_count' in metrics and metrics['trade_count'] > 0:
    print(f"\nTrade Metrics:")
    print(f"  Trade Count: {metrics.get('trade_count', 0)}")
    print(f"  Win Rate: {metrics.get('win_rate', 0):.2%}")
    print(f"  Profit Factor: {metrics.get('profit_factor', 0):.3f}")
print("=" * 60)


In [ ]:
# Display equity curve
from IPython.display import Image, display

equity_curve = result_dir / 'equity_curve.png'
if equity_curve.exists():
    display(Image(str(equity_curve)))
else:
    print("Equity curve plot not found")
